Modified: Jul 30, 2019
# Basic Calculus Operations

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import panel as pn

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)

In [ ]:
from utils import get_mro as mro, nprint
import utils as u

from vector import Vector as vec
from line import Line2d
import calculus as calc

In [ ]:
opts.defaults(
    opts.Image(colorbar=True, tools=['hover'], active_tools=['wheel_zoom'],
               padding=0.1, aspect='equal'),
    opts.Curve(tools=['hover'], padding=0.1),
)


In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

In [ ]:
# test grid class
from grid import CartesianGrid
from sdfs import sdUnitHline, sdStar1, eval_sdf
from functools import partial
from samples import LSTestSample

xs = np.linspace(-1,1,10)
ys = np.linspace(-1,1,10)[::-1]
zz = eval_sdf(xs, ys, sdStar1)

In [ ]:
xs, ys, zz = LSTestSample.linear_array()


In [ ]:
hv.Image(

In [ ]:
hv.Image((xs,ys,zz))

In [ ]:
r0 = [0]*9
r1 = [1]*9

np.r_[ [r0,r0,r0, r1,r1,r1, r0,r0,r0, r1,r1,r1] ]
np.c_[r0,r0,r0, r1,r1,r1, r0,r0,r0, r1,r1,r1 ]

In [ ]:
r = np.r_[ [r0,r1] ] ; r

In [ ]:
np.tile(r, (1,2))

In [ ]:
pattern = np.atleast_2d([0,0,0,1,1,1])
np.tile(pattern, (3,3))

In [ ]:
np.tile(pattern.T, (2,

In [ ]:
pulse = LSTestSample.pulse_grid()
test = LSTestSample.manual_grid1()

In [ ]:
pulse

In [ ]:
pulse.shape


In [ ]:
(
    hv.Image(pulse, label='x-pulse') 
    + hv.Image(pulse.T, label='y-pulse') 
    + hv.Image(test, label='test')
).opts(normalize=False).opts( opts.Image(cmap='gray') ).cols(2)

In [ ]:
dxb, dxf = calc.gradx_bf(pulse, True)
dyb, dyf = calc.grady_bf(pulse, True)

In [ ]:
bounds = (0,0,*pulse.shape)
gridstyle = {
#     'grid_line_dash': [10, 10],
    'grid_line_color': 'black', 
    'minor_grid_line_color':'lightgray',
    'grid_bounds':(0,pulse.shape[0]),
}

In [ ]:
%%opts Image (cmap='gray', alpha=0.9) [show_grid=True, gridstyle=gridstyle]
base = hv.Image(pulse, bounds=bounds, label='original') 
(
    base + hv.Image(dxb, bounds=bounds, label='dxb') 
    + base + hv.Image(dxf, bounds=bounds, label='dxf')
    + base + hv.Image(dyb, bounds=bounds, label='dyb') 
    + base + hv.Image(dyf, bounds=bounds, label='dyf')
).cols(2)

In [ ]:
calc.run_grad_tests()

In [ ]:
calc.test_diff1_bf_on_pulse_x()

In [ ]:
calc.test_diff1_central_on_pulse_x()

In [ ]:
calc.test_diff1_bf_on_linear_array()

In [ ]:
*_, linear = LSTestSample.linear_array()
dxb,dxf, dyb, dyf= calc.diff1_bf(linear[:10,:10])

In [ ]:
dyf_img = hv.Image(dyf)

In [ ]:
%opts Image (cmap='gray') {+axiswise}

dyf_img + hv.operation.histogram(dyf_img, bin_range=(-.05,.05))

In [ ]:
clipping = {'min': 'red', 'max': 'green', 'NaN': 'gray'}
temp_opts = dict(cmap='Blues_r', colorbar=True, width=300, height=230, axiswise=True)


In [ ]:
dyf_img.opts(width=500, height=500, data_aspect=1, cmap='gray', clipping_colors=clipping)

The cause of the stripe pattern is the floating point precision. Even though numpy semantically acknowledges they are really really close, thus the same number (determined by np.isclose), python's `==` will say they are not the same number. This will occur often during the finite difference operations, so let's write a function that remaps all close-enough-that-they-are-the-same values to a unqie representative number. 

- Modified: Jul 30, 2019
    - this function is in `uitls.py`, and is called 'clip_close_values'

In [ ]:
from utils import clip_close_values

In [ ]:
def linear_array():
    h,w = 100,100
    xs = np.linspace(-1,1,num=w)
    ys = np.linspace(-1,1,num=h)[::-1]
    zz = np.empty((w,h))
    for i in range(len(xs)):
        for j in range(len(ys)):
            zz[j,i] = ys[j] 
    return (xs,ys,zz)

xs,ys,zz = linear_array()

In [ ]:
plt.imshow(zz,cmap='gray')

In [ ]:
# dy, dx = np.gradient(zz)
dy,dx = calc.diff1_central(zz)

In [ ]:
plt.imshow(dy,cmap='gray')

In [ ]:
hv.Image(dy).opts(cmap='gray').hist()

Lovely!:)

In [ ]:
tiny1 = 0#0.02020202020202011
tiny2 = 1#0.020202020202020332
original = np.atleast_2d([tiny1,tiny1, tiny2, tiny2, tiny1])

In [ ]:
original = dyf
clipped = clip_close_values(original)
f,ax = plt.subplots(1,2)
ax[0].imshow(original)
ax[0].set_title('original')
ax[1].imshow(clipped)
ax[1].set_title('clipped')

Beautiful!! well done:)
Now moving on....

Next step is to check if this cleaning up fixes the levelset propagation bugs. 